In [1]:
import warnings
warnings. filterwarnings('ignore')

![title](Fig1.png)<br>
![title](Fig2.png)<br>
![title](Fig3.png)<br>

In [2]:
c = ipp.Client()
num_qubits=8
v = c.load_balanced_view([0,1,2,3,4,5,6,7])

In [3]:
location = './cachedir'
memory = Memory(location, verbose=0)

In [3]:
def egBandHamiltonianPartition(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_0=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                line=line.split()
                i,j=map(int,line[:2])
                val=float(line[2])
                if(i==j):
                    eg_h1_0[i,j]=eg_h1_0[i+num_sites,j+num_sites]=val
                else:
                    eg_h1_1[i,j]=eg_h1_1[i+num_sites,j+num_sites]=val
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1_0=eg_h1_0+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2_0=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        eg_h2_1=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            if((i==j) and (k==l)):
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k,l] = eg_h2_0[i,j,k+num_sites,l+num_sites] \
                = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
            else:
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
                eg_h2_1[i+num_sites,j+num_sites,k,l] = eg_h2_1[i,j,k+num_sites,l+num_sites] = 0.5*val
    return eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1
def qubitHamiltonianGroups(U):
    def qubitOp(h1,h2):
        fer_op=fermionic_op_builder.build_ferm_op_from_ints(h1,h2)
        mapper=JordanWignerMapper()
        qubit_op=mapper.map(fer_op)
        return qubit_op
    eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U)
    #grouping-1
    qubitH_0=qubitOp(eg_h1_1,numpy.zeros((8,8,8,8)))
    qubitH_1=qubitOp(eg_h1_0,eg_h2_0)
    qubitH_2=qubitOp(numpy.zeros((8,8)),eg_h2_1)
    #grouping-2
#     qubitH_0=qubitOp(eg_h1_1+eg_h1_0,numpy.zeros((8,8,8,8)))
#     qubitH_1=qubitOp(numpy.zeros((8,8)),eg_h2_0)
#     qubitH_2=qubitOp(numpy.zeros((8,8)),eg_h2_1)
    return qubitH_0,qubitH_1,qubitH_2

In [4]:
def OffDiag2BodyTerms(Op,send_inverse=False):
    theta=numpy.pi/4
    #Prepare UnitaryOp for rotating Hamilotnian terms
    O_0_1_up=(numpy.cos(theta/2.)*I^I^I^I^I^I^I^I)+(1j*numpy.sin(theta/2.)*I^I^I^I^I^I^X^Y)
    O_1_0_up=(numpy.cos(theta/2.)*I^I^I^I^I^I^I^I)+(-1j*numpy.sin(theta/2.)*I^I^I^I^I^I^Y^X)
    O_0_1_down=(numpy.cos(theta/2.)*I^I^I^I^I^I^I^I)+(1j*numpy.sin(theta/2.)*I^I^X^Y^I^I^I^I)
    O_1_0_down=(numpy.cos(theta/2.)*I^I^I^I^I^I^I^I)+(-1j*numpy.sin(theta/2.)*I^I^Y^X^I^I^I^I)
    O_01=O_0_1_up@O_1_0_up@O_0_1_down@O_1_0_down
    O=O_01.reduce()
    Odag=O.adjoint()
    rotH=O@Op@Odag
    rotH=rotH.reduce().reduce()
     #theta=numpy.pi/4
    #Prepare gate implementing Givens Rotn for appending to ansatz circ pre-measurement
    RotMat=(1/numpy.sqrt(2))*numpy.array([[1,-1],[1,1]])
    w,v=numpy.linalg.eigh(RotMat)
    givens=givensRotns(v.T)[0]
    circ=QuantumCircuit(2)
    for rots in givens:
        for tup in rots:
            circ.cnot(tup[1],tup[0])
            circ.cry(-2*tup[2],tup[0], tup[1])
            circ.cnot(tup[1], tup[0])
            circ.rz(tup[3],tup[1])        
    circ0_1gate=circ.to_instruction()
    if send_inverse==True:
        circ2=circ.inverse()
        circ0_1gate_inverse=circ2.to_instruction()   
        return circ0_1gate,circ0_1gate_inverse,rotH
    else:
        return circ0_1gate,rotH
def OffDiag1BodyTerms(Op,send_inverse=False):
    def qubitOp(h1,h2):
        fer_op=fermionic_op_builder.build_ferm_op_from_ints(h1,h2)
        mapper=JordanWignerMapper()
        qubit_op=mapper.map(fer_op)
        return qubit_op
    h1=eg_h1_1#+eg_h1_0
#     Op=qubitOp(h1,numpy.zeros((8,8,8,8)))
    Op=Op.reduce()
    w,v=numpy.linalg.eigh(h1)
    givens=givensRotns(v.T)[0]
    OpNew=qubitOp(numpy.diag(w),numpy.zeros((8,8,8,8)))
    #Prepare gate implementing Givens Rotn for appending to ansatz circ pre-measurement
    circ=QuantumCircuit(8)
    for rots in givens:
        for tup in rots:
            circ.cnot(tup[1],tup[0])
            circ.cry(-2*tup[2],tup[0], tup[1])
            circ.cnot(tup[1], tup[0])
            circ.rz(tup[3],tup[1])     
    circ0_8gate=circ.to_instruction()
    if send_inverse==True:
        circ2=circ.inverse()        
        circ0_8gate_inverse=circ2.to_instruction()
        return circ0_8gate,circ0_8gate_inverse,OpNew
    else:
        return circ0_8gate,OpNew

In [5]:
def construct_dataForm(data):
    q_instance,qr,backend,circ0_8gate,circ0_1gate,circ,op,i=data
    circ1=circ.copy()
    if i==0:
        circ1.append(circ0_8gate,qr)
    elif i==2:
        circ1.append(circ0_1gate,[qr[0],qr[1]])
        circ1.append(circ0_1gate,[qr[4],qr[5]])         
    psi = CircuitStateFn(circ1)
    # define the state to sample
    measurable_expression = StateFn(op, is_measurement=True).compose(psi) 
    # convert to expectation value
    expectation = PauliExpectation(group_paulis=True).convert(measurable_expression) 
    return expectation

In [1]:
def NoiseModel(p_meas=0.01,prob1=0.001,prob2=0.01):
    # Error probabilities
    #measurement error
    error_meas = noise.pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    #prob_1 - 1-qubit gate
    #prob_2 - 2-qubit gate
    # Depolarizing quantum errors
    error_1 = noise.depolarizing_error(prob1, 1)
    error_2 = noise.depolarizing_error(prob2, 2)
    # Add errors to noise model
    noise_model = noise.NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure")
    noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
    noise_model.add_all_qubit_quantum_error(error_2, ['cx'])
    # Get basis gates from noise model
    print(noise_model)
    basis_gates = noise_model.basis_gates
    return noise_model,basis_gates

In [6]:
def EstmTerms(data):
    from qiskit.opflow import PauliExpectation, CircuitSampler
    import time
    t1=time.time()
    q_instance,expectation=data
    sampler = CircuitSampler(q_instance).convert(expectation)
    var=PauliExpectation().compute_variance(sampler)
    O_sampled=sampler.eval()
    del expectation
    return O_sampled,var,time.time()-t1

In [2]:
def HamiltonianEstm(params):
    time1=time.time()
    backend = AerSimulator(method='density_matrix')
    num_shots_arr=[2**18,2**21,2**20]
    qc=var_form_base.construct_circuit(parameters=params)
    q_instance_arr=[QuantumInstance(backend,shots=num_shots_arr[i],basis_gates=basis_gates,noise_model=noise_model) for  i in range(len(num_shots_arr))]
    data_arr=[(q_instance_arr[i],qr,backend,circ0_8gate,circ0_1gate,qc,Ops[i],i) for i in range(len(Ops))]
    inp_data=[(q_instance_arr[i],construct_dataForm(data_arr[i])) for i in range(len(data_arr))]
    result = v.map_async(EstmTerms, inp_data)
    output=result.get()
    Mean=numpy.real(output[0][0]+output[1][0]+output[2][0])
    indMean=numpy.array([numpy.real(output[0][0]),numpy.real(output[1][0]),numpy.real(output[2][0])])
    indErr=numpy.array([numpy.sqrt(output[0][1]/num_shots_arr[0]),numpy.sqrt(output[1][1]/num_shots_arr[1]),numpy.sqrt(output[2][1]/num_shots_arr[2])])
    indTimes=numpy.array([output[0][2],output[1][2],output[2][2]])
    netErr=indErr[0]+indErr[1]+indErr[2]
    v.purge_results('all')
    del data_arr
    del inp_data
    state_vec=execute(qc,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()
    exact=state_vec@op_converter.to_matrix_operator(qubitH).dense_matrix@numpy.conjugate(state_vec)
    print("mean",Mean,"abs err",numpy.abs(Mean-exact),exact)
    return Mean,netErr.real,indMean,indErr,indTimes
def HamiltonianEstmFrugal(params,num_shots_arr=[2**14,2**15,2**15],ret_err=False):
    time1=time.time()
    backend = AerSimulator(method='density_matrix')
    qc=var_form_base.construct_circuit(parameters=params)
    q_instance_arr=[QuantumInstance(backend,shots=num_shots_arr[i],basis_gates=basis_gates,noise_model=noise_model) for  i in range(len(num_shots_arr))]
    data_arr=[(q_instance_arr[i],qr,backend,circ0_8gate,circ0_1gate,qc,Ops[i],i) for i in range(len(Ops))]
    inp_data=[(q_instance_arr[i],construct_dataForm(data_arr[i])) for i in range(len(data_arr))]
    result = v.map_async(EstmTerms, inp_data)
    output=result.get()
    Mean=numpy.real(output[0][0]+output[1][0]+output[2][0])
    indMean=numpy.array([numpy.real(output[0][0]),numpy.real(output[1][0]),numpy.real(output[2][0])])
    indErr=numpy.array([numpy.sqrt(output[0][1]/num_shots_arr[0]),numpy.sqrt(output[1][1]/num_shots_arr[1]),numpy.sqrt(output[2][1]/num_shots_arr[2])])
    indTimes=numpy.array([output[0][2],output[1][2],output[2][2]])
    netErr=indErr[0]+indErr[1]+indErr[2]
    v.purge_results('all')
    del data_arr
    del inp_data
    state_vec=execute(qc,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()
    exact=state_vec@op_converter.to_matrix_operator(qubitH).dense_matrix@numpy.conjugate(state_vec)
    if ret_err==False:
        return Mean
    elif ret_err==True:
        return Mean,netErr.real,indMean,indErr,indTimes#,exact

In [1]:
def estimate(PauliOp,counts):
    PauliString,coeff=PauliOp.primitive.to_list()[0]
    indexOfZs=numpy.where(numpy.array(list(PauliString))=='Z')[0]
    bitstrings,cnts=list(counts. keys()),list(counts.values())
    p=0
    for i in range(len(bitstrings)):
        counter_1=list(numpy.array(list(bitstrings[i]))[indexOfZs]).count('1')
        if counter_1%2==1:
            p=p+cnts[i]
    p=p/sum(cnts)
    mean=(1-2*p)
    return mean*coeff.real
def meanOp(PauliOp,counts):
    mean=0
    for i in range(len(PauliOp)):
        mean=mean+estimate(PauliOp[i],counts)
    return mean    
def covariance(Op,i,j,counts):
    cov=estimate(Op[i]@Op[j],counts)-estimate(Op[i],counts)*estimate(Op[j],counts)
    return cov
def covbtnOps(A,B,counts):
    corr=0
    for i in range(len(A)):
        for j in range(len(B)):
            corr=corr+estimate(A[i]@B[j],counts)-estimate(A[i],counts)*estimate(B[j],counts)
    return corr        
def variance(Op,counts):
    var=0
    cov_mat=numpy.zeros((len(Op),len(Op)))
    for i in range(len(Op)):
        for j in range(i,len(Op)):
            if i==j:
                mean=estimate(Op[i],counts)
                coeff=Op[i].primitive.to_list()[0][1].real
                cov_mat[i][j]=(coeff**2-mean**2)
                var=var+(coeff**2-mean**2)
            elif i!=j:
                var=var+2*covariance(Op,i,j,counts)
                cov_mat[i][j]=cov_mat[j][i]=covariance(Op,i,j,counts)
    return var
def determineStatisticOfCtrlOp(data,basis_gates=None,noise_model=None):
    CtrlOp,circ,numShots,seed,getCounts,meanOp=data
    counts=getCounts((circ,numShots,seed),basis_gates=basis_gates,noise_model=noise_model)
    mean=meanOp(CtrlOp,counts)
    return mean
def controlVariate(data):
    Op,CtrlOp,counts,meanCtrlOp=data
    cov1=covbtnOps(Op,CtrlOp,counts)
    var1=variance(CtrlOp,counts)
    coeff=-cov1/var1
    newOp=Op+((coeff)*(CtrlOp-(meanCtrlOp*I^I^I^I^I^I^I^I)))
    m=meanOp(newOp,counts)
    var=variance(newOp,counts)
    err=numpy.sqrt(var/sum(counts.values()))
    return m.real,err.real
def getCounts(Input,basis_gates=None,noise_model=None):
    circ,num_shots,seed=Input
    circ1=circ.copy()
    circ1.add_register(ClassicalRegister(8,'c'))
    circ1.measure([0,1,2,3,4,5,6,7],[0,1,2,3,4,5,6,7]) 
    if seed==None:
        job_sim = execute(circ1, AerSimulator(method='density_matrix'), shots=num_shots,basis_gates=basis_gates,noise_model=noise_model)
        result_sim = job_sim.result()
        counts = result_sim.get_counts(circ1) 
    else:
        job_sim = execute(circ1, AerSimulator(method='density_matrix'), shots=num_shots,seed_simulator=seed,basis_gates=basis_gates,noise_model=noise_model)
        result_sim = job_sim.result()
        counts = result_sim.get_counts(circ1)
    return counts
def Hestimate(params):
    #Main circuit associated with the present Hamiltonian being studied
    circ=var_form_base.construct_circuit(parameters=params)
    circ_arr=[circ.copy(),circ.copy(),circ.copy()]
    circ_arr[0].append(circ0_8gate,qr)
    circ_arr[2].append(circ0_1gate,[qr[0],qr[1]])
    circ_arr[2].append(circ0_1gate,[qr[4],qr[5]])
    counts_Arr=Parallel(n_jobs=6,verbose=2)(delayed(getCounts)((circuit,2**18,None),basis_gates=basis_gates,noise_model=noise_model) for circuit in circ_arr)
    #Control Hamiltonian 
    circ_arr_ctrl=[circ.copy(),circ.copy(),circ.copy()]
    circ_arr_ctrl[0].append(circ0_8gate_ctrl,qr)
    circ_arr_ctrl[2].append(circ0_1gate_ctrl,[qr[0],qr[1]])
    circ_arr_ctrl[2].append(circ0_1gate_ctrl,[qr[4],qr[5]])
    inp_data=[]
    Num=30
    shots=2**12
    for i in range(len(Ops_ctrl)):
        for seed in range(Num):
            inp_data.append((Ops_ctrl[i],circ_arr_ctrl[i],shots,seed,getCounts,meanOp))
    res=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(determineStatisticOfCtrlOp)(dat,basis_gates=basis_gates,noise_model=noise_model) for dat in inp_data))
    #mean0,var,std=stats.bayes_mvs(res[:Num],alpha=0.95)
    #mean1,var,std=stats.bayes_mvs(res[Num:2*Num],alpha=0.95)
    #mean2,var,std=stats.bayes_mvs(res[2*Num:],alpha=0.95)
    mean_res=numpy.mean(res[:Num]),numpy.mean(res[Num:2*Num]),numpy.mean(res[2*Num:]) #mean0.statistic,mean1.statistic,mean2.statistic#res[0],res[1],res[2]#
    dataTrain=[(Ops[i],Ops_ctrl[i],counts_Arr[i],mean_res[i]) for i in range(3)]
    m1,err1=controlVariate(dataTrain[0])
    m2,err2=controlVariate(dataTrain[1])
    m3,err3=controlVariate(dataTrain[2])
    m=m1+m2+m3
    err=err1+err2+err3
    #exact result from statevector just for checking
    state_vec=execute(circ,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()
    exact=state_vec@op_converter.to_matrix_operator(qubitH).dense_matrix@numpy.conjugate(state_vec)
    #print("mean",m,"abs error",numpy.abs(m-exact))
    #print("**********\n")
    return m,err,(m1,m2,m3),(err1,err2,err3),exact

In [8]:
def OpEstm(Ops,params,i,num_shots):
    time1=time.time()
    backend = AerSimulator(method='density_matrix')
    #num_shots=2**10
    qc=var_form_base.construct_circuit(parameters=params)
    data=(QuantumInstance(backend,shots=num_shots),qr,backend,circ0_8gate,circ0_1gate,qc,Ops[i],i)
    inp_data=(QuantumInstance(backend,shots=num_shots),construct_dataForm(data))
    output = EstmTerms(inp_data)
    Mean=numpy.real(output[0])
    var=output[1]
    print("time elapsed",time.time()-time1)
    return Mean.real,numpy.sqrt(var.real/num_shots),#Times#,output[3]

In [9]:
def directHamMeasure(params):
    backend=Aer.get_backend('qasm_simulator')
    time1=time.time()
    qc=var_form_base.construct_circuit(parameters=params)
    psi = CircuitStateFn(qc)
    # define the state to sample
    measurable_expression = StateFn(qubitH, is_measurement=True).compose(psi) 
    # convert to expectation value
    expectation = PauliExpectation(group_paulis=True).convert(measurable_expression) 
    sampler = CircuitSampler(QuantumInstance(backend,shots=num_shots)).convert(expectation)
    var=PauliExpectation().compute_variance(sampler)/num_shots
    O_sampled=sampler.eval()
    print(time.time()-time1)
    return O_sampled,numpy.sqrt(var),time.time()-time1

In [1]:
def RotatedHamiltonianToDiagonalGroups(U):
    #Hamiltonian grouping for circuit based measurement
    eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U)
    qubitH_0,qubitH_1,qubitH_2=qubitHamiltonianGroups(U)
    opArr=[qubitH_0,qubitH_1,qubitH_2]
    #two body off diags
    circ0_1gate,rotH2body=OffDiag2BodyTerms(opArr[2])
    #one body off diags
    circ0_8gate,rotH1body=OffDiag1BodyTerms(opArr[0])
    Ops=[rotH1body,opArr[1],rotH2body]
    return Ops

__Two electron interaction__<br>
$h_{ijkl}c^{\dagger}_{i}c^{\dagger}_{k}c_{l}c_{j} =\underbrace{\sum_{a}\mathcal{L}_{ij}^{a}\mathcal{L}^{a}_{kl}c^{\dagger}_{i}c^{\dagger}_{k}c_{l}c_{j}}_{Cholesky   Decomposition}=\underbrace{\sum_{a,p,q}U_{ip}^{a}\lambda^{a}_{p}U_{pj}^{*}U_{kq}^{a}\lambda^{a}_{q}U_{ql}^{*}c^{\dagger}_{i}c^{\dagger}_{k}c_{l}c_{j}}_{\text{diagonalizing Cholesky matrices leading to eigenvalues}}$<br>

$c^{\dagger}_{i}=\sum_{m}O_{im}f^{\dagger}_{m}$ (__single particle basis rotn. from AO to MO basis__)

__Putting it into the two electron interaction terms__

$\sum_{ijkl}h_{ijkl}c^{\dagger}_{i}c^{\dagger}_{k}c_{l}c_{j}=\sum_{a}\sum_{p,q}\sum_{ijkl,mnm'n'}U_{ip}^{a}\lambda^{a}_{p}U_{pj}^{*}U_{kq}^{a}\lambda^{a}_{q}U_{ql}^{*}O_{im}O_{km}O^{*}_{ln'}O^{*}_{jm'}f^{\dagger}_{m}f^{\dagger}_{n}f_{n'}f_{m'}$
$=\sum_{a}\sum_{p,q}\lambda^{a}_{p}\lambda^{a}_{q}\hat{n}^{(a)}_{p}\hat{n}^{(b)}_{q}$<br>

$\hat{n}^{(a)}_{p}=g^{a,\dagger}_{p}g^{a}_{p}$<br>
$g^{a,\dagger}_{p}=\sum_{i,m}U_{ip}^{a}O_{im}f^{\dagger}_{m}=\sum_{i,m}\left[[U^{a}]^{T}O\right]_{pm}f^{\dagger}_{m}$<br>

Thus proved that __$\lambda^{a}_{p}$'s remains invariant__ under single particle basis rotation, only the circuits will change because the Given's rotation circuits need to be appended with the AO to MO basis rotation components, i.e.

#### AO to MO basis transformation rule: Replace earlier Given Rotation circuits $U^{a}$  with $U^{a}O $, where $O$ is the transformation from AO to MO.


In [11]:
import numpy
x=numpy.array([1,0])
x += x == 0

In [12]:
x

array([1, 1])